In [111]:
import keras as k
import keras.backend as b
import keras.layers as l
import numpy as np
import sys
import theano as t
sys.path.append('../src/mane/')

In [2]:
import graph as g
fb = g.graph_from_pickle('../src/mane/data/egonets.graph')

In [60]:
t_in = l.Input(batch_shape=(100,1), dtype='int32', name='target')

In [61]:
c_in = l.Input(batch_shape=(100,1), dtype='int32', name='class')

In [79]:
emb_t = l.Embedding(input_dim=len(fb), output_dim=200, name='target_emb', input_length=1)(t_in)
emb_t = l.Reshape(target_shape=(200,))(emb_t)
emb_c = l.Embedding(input_dim=len(fb), output_dim=200, name='class_emb', input_length=1)(c_in)
emb_c = l.Reshape(target_shape=(200,))(emb_c)

In [80]:
def row_dot(inputs):
    return b.batch_dot(inputs[0], inputs[1], axes=[1,1])

In [86]:
dot_prod = l.Merge(mode=row_dot, output_shape=(1,), name='score')([emb_t, emb_c])

In [127]:
model = k.models.Model(input=[t_in, c_in], output=dot_prod)

In [125]:
def nce_loss(y_true, y_pred):
    y_true = y_true.reshape(y_pred.shape)
    return -b.log(b.sigmoid(y_pred * y_true))

In [128]:
model.compile(loss=nce_loss, optimizer='adam', name='mmodel')

In [129]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
class (InputLayer)               (100, 1)              0                                            
____________________________________________________________________________________________________
target (InputLayer)              (100, 1)              0                                            
____________________________________________________________________________________________________
class_emb (Embedding)            (100, 1, 200)         807800      class[0][0]                      
____________________________________________________________________________________________________
target_emb (Embedding)           (100, 1, 200)         807800      target[0][0]                     
___________________________________________________________________________________________

In [135]:
model.evaluate(x={'target':np.array([[3]],dtype=np.int32), 'class':np.array([[5]], dtype=np.int32)}, y=np.array([1], dtype=np.float32))

1/1 [==============================] - 0s


0.69442957639694214

In [112]:
model.predict(x={'target':np.array([[12],[41]],dtype=np.int32), 'class':np.array([[3],[4]], dtype=np.int32)})

array([[-0.00083925],
       [ 0.01775739]], dtype=float32)

In [115]:
pred = model.predict(x={'target':np.array([[1],[2]],dtype=np.int32), 'class':np.array([[3],[4]], dtype=np.int32)})

In [116]:
true = np.array([1.0,1.0], dtype=np.float32)

In [126]:
nce_loss(true, pred).eval()

array([[ 0.69689059],
       [ 0.70109516]], dtype=float32)

In [118]:
i = model.get_weights()[0]

In [119]:
o = model.get_weights()[1] 

In [122]:
i[0]

array([  5.37328469e-03,   3.56331877e-02,   2.62254160e-02,
         4.51384634e-02,   3.32368687e-02,   4.87604439e-02,
        -3.27504389e-02,  -2.82911789e-02,  -2.18070131e-02,
        -9.18343943e-03,  -2.35433318e-03,   2.02998202e-02,
        -1.89829934e-02,  -4.47647609e-02,  -3.23314639e-03,
        -3.53223495e-02,  -2.69703176e-02,  -2.33573802e-02,
         4.06676419e-02,   3.85162197e-02,   3.40460762e-02,
         3.68706770e-02,   3.97941098e-03,  -4.44951616e-02,
         3.56409624e-02,   1.95348752e-03,   2.54548322e-02,
         2.65211426e-02,   1.32009741e-02,   1.87109550e-03,
        -1.68678928e-02,  -1.03147337e-02,   1.99237075e-02,
        -2.24945471e-02,   4.42732945e-02,   2.67329495e-02,
        -4.30680700e-02,  -4.43621501e-02,   4.84206602e-02,
         1.39703155e-02,   2.25176290e-02,  -4.67611142e-02,
        -1.74244680e-02,  -9.13463626e-03,   1.77482497e-02,
         3.53569351e-02,   3.68807465e-02,  -4.16654907e-02,
         3.04941386e-02,